In [ ]:
#Auto-reload modules (used to develop functions outside this notebook)
%load_ext autoreload
%autoreload 2

In [ ]:
from labrotation.two_photon_session import TwoPhotonSession
import labrotation.file_handling as fh
from matplotlib import pyplot as plt
import os
import h5py

In [ ]:
fname = fh.open_file("Open h5 file!")

In [ ]:
print(fname)

In [ ]:
session = TwoPhotonSession.from_hdf5(fname)

In [ ]:
with h5py.File(fname) as hf:
    fluo_improved = hf["mean_fluo_nucleus"][()]
    binary_img = hf["selected_pixels"][()]
    fov = hf["fov"][()]

In [ ]:
session.time_offs_lfp_nik

In [ ]:
if type(session.ND2_TIMESTAMPS_PATH) == bytes:
    session.ND2_TIMESTAMPS_PATH = session.ND2_TIMESTAMPS_PATH.decode()

In [ ]:
session.nikon_meta

In [ ]:
session._load_nikon_meta()

In [ ]:
session.nikon_meta = session.nikon_meta[session.nikon_meta["Events Type"] != "Stimulation"]

In [ ]:
nik_sw_t = session.nikon_meta["SW Time [s]"].values
nik_t_approx_s = nik_sw_t - nik_sw_t[0]

In [ ]:
lfp_mov_t, lfp_mov_y = session.lfp_movement()
lfp_lfp_t, lfp_lfp_y = session.lfp_lfp()

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(18,18), sharex=True)
axs[0].plot(nik_t_approx_s, session.mean_fluo, color="green")
axs[1].plot(lfp_lfp_t, lfp_lfp_y, color="blue")
axs[2].plot(lfp_mov_t, lfp_mov_y, color="grey")

axs[2].set_xlim((20, 25))
plt.show()

In [ ]:
export_dict = {"fluo_y":session.mean_fluo, "fluo_y_nucleus":fluo_improved, "fluo_t":nik_t_approx_s, "lfp_y":lfp_lfp_y, "lfp_t":lfp_lfp_t, "loco_t":lfp_mov_t, "loco_y":lfp_mov_y, "selected_pixels":binary_img, "fov":fov}

In [ ]:
out_fname = os.path.splitext(os.path.split(fname)[1])[0] + "_matlabized.h5"
out_fpath = os.path.join(os.path.split(fname)[0], out_fname)
print(out_fpath)

In [ ]:
save_file = False
if save_file:
    with h5py.File(out_fpath, "w") as hf:
        for key in export_dict.keys():
            hf[key] = export_dict[key]
    print("Saved file.")
else:
    print("File not saved!")